In [1]:
import pandas as pd
import numpy as np
import time

import datetime as dt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter

import warnings
warnings.filterwarnings("ignore")


from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
%matplotlib inline
import seaborn as sns

In [2]:
#Importing the data
df = pd.read_csv('online_retail-2010-2011.csv')
df=df.copy()

In [3]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
#UK müşterilerinin filtrelenmesi
uk_df=df[df.Country=='United Kingdom']
uk_df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [5]:
uk_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495478 entries, 0 to 541893
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      495478 non-null  object 
 1   StockCode    495478 non-null  object 
 2   Description  494024 non-null  object 
 3   Quantity     495478 non-null  int64  
 4   InvoiceDate  495478 non-null  object 
 5   Price        495478 non-null  float64
 6   Customer ID  361878 non-null  float64
 7   Country      495478 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 34.0+ MB


In [6]:
uk_df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [7]:
#İade edilen ürünlerin droplanması
df.dropna(inplace=True)
df = df[~df["Invoice"].str.contains("C", na=False)]
df = df[df["Quantity"] > 0]
#Total Price sütununun oluşturulması
df['TotalPrice'] = df['Price'] * df['Quantity']

uk_df.reset_index(inplace = True)

In [8]:
#CLTV için ilgili sütunların seçilmesi
uk_df=uk_df[['Customer ID','InvoiceDate','Invoice','Quantity','Price']]
uk_df

,Customer ID,InvoiceDate,Invoice,Quantity,Price
0,17850.0,12/1/2010 8:26,536365,6,2.55
1,17850.0,12/1/2010 8:26,536365,6,3.39
2,17850.0,12/1/2010 8:26,536365,8,2.75
3,17850.0,12/1/2010 8:26,536365,6,3.39
4,17850.0,12/1/2010 8:26,536365,6,3.39
...,...,...,...,...,...
495473,15804.0,12/9/2011 12:31,581585,12,1.95
495474,13113.0,12/9/2011 12:49,581586,8,2.95
495475,13113.0,12/9/2011 12:49,581586,24,1.25
495476,13113.0,12/9/2011 12:49,581586,24,8.95


In [9]:
def comma_to_dot(x):
    x=x.replace(',','.')
    return x

#uk_df['Price']=uk_df['Price'].apply(comma_to_dot)

uk_df['Price']=uk_df['Price'].astype("float64")

uk_df['TotalPrice'] = uk_df['Quantity'] * uk_df['Price']

In [10]:
# CLTV için tarihin belirlenmesi
today_date = dt.datetime(2011, 12, 11)

In [11]:
uk_df['InvoiceDate']= uk_df['InvoiceDate'].astype('datetime64[ns]')

In [12]:
#recency,Tenure,frequency, monetary değerlerinin hesaplanması
cltv_df = uk_df.groupby('Customer ID').agg({'InvoiceDate': [lambda date: (date.max() - date.min()).days,
                                                                    lambda date: (today_date - date.min()).days],
                                                    'Invoice': lambda num: num.nunique(),
                                                    'TotalPrice': lambda TotalPrice: TotalPrice.sum()})
cltv_df.columns = cltv_df.columns.droplevel(0)

In [13]:
cltv_df

,<lambda_0>,<lambda_1>,<lambda>,<lambda>
Customer ID,,,,
12346.0,0,326,2,0.00
12747.0,366,370,11,4196.01
12748.0,372,374,224,29072.10
12749.0,209,214,8,3868.20
12820.0,323,327,4,942.34
...,...,...,...,...
18280.0,0,278,1,180.60
18281.0,0,181,1,80.82
18282.0,118,127,3,176.60


In [14]:
#sütunların isimlendirilmesi
cltv_df.columns = ['recency', 'T', 'frequency', 'monetary']

In [15]:
cltv_df

,recency,T,frequency,monetary
Customer ID,,,,
12346.0,0,326,2,0.00
12747.0,366,370,11,4196.01
12748.0,372,374,224,29072.10
12749.0,209,214,8,3868.20
12820.0,323,327,4,942.34
...,...,...,...,...
18280.0,0,278,1,180.60
18281.0,0,181,1,80.82
18282.0,118,127,3,176.60


In [16]:
#monetary değerinin 0'dan büyük olanların seçilmesi
cltv_df = cltv_df[cltv_df["monetary"] > 0]
cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"]
# frequency ve recency değerlerinin haftalık olarak hesaplanması
cltv_df['recency'] = cltv_df['recency'] / 7
cltv_df['T'] = cltv_df['T'] / 7
#Frekans değerlerinden 1'den büyük olanların seçilmesi
cltv_df = cltv_df[cltv_df['frequency'] > 1]
cltv_df

,recency,T,frequency,monetary
Customer ID,,,,
12747.0,52.285714,52.857143,11,381.455455
12748.0,53.142857,53.428571,224,129.786161
12749.0,29.857143,30.571429,8,483.525000
12820.0,46.142857,46.714286,4,235.585000
12822.0,2.285714,12.571429,3,306.326667
...,...,...,...,...
18276.0,3.142857,6.285714,3,107.786667
18277.0,37.142857,45.571429,2,48.815000
18282.0,16.857143,18.142857,3,58.866667


In [17]:
#BG-NBD modelin kurulması
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df['frequency'], cltv_df['recency'], cltv_df['T'])

<lifetimes.BetaGeoFitter: fitted with 2743 subjects, a: 0.19, alpha: 7.88, b: 2.81, r: 1.93>

In [18]:
#6 aylık beklenen tahmini satın alımın hesaplanması
bgf.conditional_expected_number_of_purchases_up_to_time(24, 
                                                 # 6 months = 180 days
                                                        cltv_df['frequency'],
                                                        cltv_df['recency'],
                                                        cltv_df['T']).sort_values(ascending=False).head(10)

Customer ID
12748.0    85.399732
17841.0    64.528741
14606.0    49.099077
13089.0    45.632853
15311.0    45.346467
14096.0    35.015606
12971.0    34.323566
14527.0    33.184002
13408.0    31.294039
16422.0    27.169732
dtype: float64

In [19]:
#GAMMA-GAMMA modelinin kurulması
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df['frequency'], cltv_df['monetary'])
cltv_df["expected_average_profit"] = ggf.conditional_expected_average_profit(cltv_df['frequency'], cltv_df['monetary'])

In [20]:
#6 aylık CLTV tahmininin yapılması
month=6

cltv_df['Six_Month_CLTV_Pred']= ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=month,  # aylık
                                   freq="W",  # T'nin frekans bilgisi.
                                   discount_rate=0.01)

cltv_df

,recency,T,frequency,monetary,expected_average_profit,Six_Month_CLTV_Pred
Customer ID,,,,,,
12747.0,52.285714,52.857143,11,381.455455,390.508852,1987.847159
12748.0,53.142857,53.428571,224,129.786161,129.944986,11618.283215
12749.0,29.857143,30.571429,8,483.525000,499.314594,3012.353356
12820.0,46.142857,46.714286,4,235.585000,251.973652,640.769451
12822.0,2.285714,12.571429,3,306.326667,334.974942,838.277644
...,...,...,...,...,...,...
18276.0,3.142857,6.285714,3,107.786667,118.727014,819.501207
18277.0,37.142857,45.571429,2,48.815000,57.733662,93.634578
18282.0,16.857143,18.142857,3,58.866667,65.443803,276.714027


In [21]:
#1 aylık CLTV hesaplanması
month=1

cltv_df['One_Month_CLTV_Pred']= ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=month,  # aylık
                                   freq="W",  # T'nin frekans bilgisi.
                                   discount_rate=0.01)

cltv_df

,recency,T,frequency,monetary,expected_average_profit,Six_Month_CLTV_Pred,One_Month_CLTV_Pred
Customer ID,,,,,,,
12747.0,52.285714,52.857143,11,381.455455,390.508852,1987.847159,349.391791
12748.0,53.142857,53.428571,224,129.786161,129.944986,11618.283215,2041.860658
12749.0,29.857143,30.571429,8,483.525000,499.314594,3012.353356,536.489931
12820.0,46.142857,46.714286,4,235.585000,251.973652,640.769451,112.914411
12822.0,2.285714,12.571429,3,306.326667,334.974942,838.277644,153.119225
...,...,...,...,...,...,...,...
18276.0,3.142857,6.285714,3,107.786667,118.727014,819.501207,152.535496
18277.0,37.142857,45.571429,2,48.815000,57.733662,93.634578,16.507282
18282.0,16.857143,18.142857,3,58.866667,65.443803,276.714027,50.002653


In [22]:
#12 aylık CLTV hesaplanması
month=12

cltv_df['One_Year_CLTV_Pred']= ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=month,  # aylık
                                   freq="W",  # T'nin frekans bilgisi.
                                   discount_rate=0.01)

cltv_df

,recency,T,frequency,monetary,expected_average_profit,Six_Month_CLTV_Pred,One_Month_CLTV_Pred,One_Year_CLTV_Pred
Customer ID,,,,,,,,
12747.0,52.285714,52.857143,11,381.455455,390.508852,1987.847159,349.391791,3756.031314
12748.0,53.142857,53.428571,224,129.786161,129.944986,11618.283215,2041.860658,21954.528347
12749.0,29.857143,30.571429,8,483.525000,499.314594,3012.353356,536.489931,5636.248562
12820.0,46.142857,46.714286,4,235.585000,251.973652,640.769451,112.914411,1208.283147
12822.0,2.285714,12.571429,3,306.326667,334.974942,838.277644,153.119225,1544.187832
...,...,...,...,...,...,...,...,...
18276.0,3.142857,6.285714,3,107.786667,118.727014,819.501207,152.535496,1495.153560
18277.0,37.142857,45.571429,2,48.815000,57.733662,93.634578,16.507282,176.504944
18282.0,16.857143,18.142857,3,58.866667,65.443803,276.714027,50.002653,512.908234


In [23]:
#1 aylık CLTV'de en yüksek olan 10 kişi
cltv_df['One_Month_CLTV_Pred'].sort_values(ascending=False).head(10)

Customer ID
18102.0    18693.137071
17450.0    13627.192417
14096.0    11264.868872
16000.0    10481.646074
17511.0     6396.234516
16684.0     5048.552180
14088.0     4528.684727
13694.0     4498.671529
15311.0     4210.447767
13089.0     4092.922380
Name: One_Month_CLTV_Pred, dtype: float64

In [24]:
#1 yıllık CLTV'de en yüksek olan 10 kişi
cltv_df['One_Year_CLTV_Pred'].sort_values(ascending=False).head(10)

Customer ID
18102.0    200874.875146
17450.0    146437.571639
14096.0    114282.204214
16000.0     98194.799169
17511.0     68777.347671
16684.0     54201.796493
14088.0     48421.746760
13694.0     48372.527924
15311.0     45272.224390
13089.0     43990.352071
Name: One_Year_CLTV_Pred, dtype: float64

In [25]:
#Müşterilerin 4 segmente ayrılması
cltv_df["Segment_Six_Month_CLTV_Pred"] = pd.qcut(cltv_df["Six_Month_CLTV_Pred"], 4, labels = [0, 1 , 2, 3])

In [26]:
cltv_df

,recency,T,frequency,monetary,expected_average_profit,Six_Month_CLTV_Pred,One_Month_CLTV_Pred,One_Year_CLTV_Pred,Segment_Six_Month_CLTV_Pred
Customer ID,,,,,,,,,
12747.0,52.285714,52.857143,11,381.455455,390.508852,1987.847159,349.391791,3756.031314,3
12748.0,53.142857,53.428571,224,129.786161,129.944986,11618.283215,2041.860658,21954.528347,3
12749.0,29.857143,30.571429,8,483.525000,499.314594,3012.353356,536.489931,5636.248562,3
12820.0,46.142857,46.714286,4,235.585000,251.973652,640.769451,112.914411,1208.283147,1
12822.0,2.285714,12.571429,3,306.326667,334.974942,838.277644,153.119225,1544.187832,1
...,...,...,...,...,...,...,...,...,...
18276.0,3.142857,6.285714,3,107.786667,118.727014,819.501207,152.535496,1495.153560,1
18277.0,37.142857,45.571429,2,48.815000,57.733662,93.634578,16.507282,176.504944,0
18282.0,16.857143,18.142857,3,58.866667,65.443803,276.714027,50.002653,512.908234,0


In [28]:
# Müşteri segmentlerinin Recency, Frequency ve Monetary bazında betimsel istatistikleri
cltv_df.groupby('Segment_Six_Month_CLTV_Pred').agg({
'recency': ['median','mean','std'],
'frequency': ['median','mean','std'],
'monetary': ['median','mean','std'],
'expected_average_profit': ['median','mean','std'] })

recency                       frequency  \
                                median       mean        std    median   
Segment_Six_Month_CLTV_Pred                                              
0                            14.500000  17.646814  15.115320       2.5   
1                            32.357143  30.489171  14.552169       4.0   
2                            32.714286  30.562044  15.849075       5.0   
3                            35.071429  31.084965  18.461206       8.0   

                                                     monetary              \
                                  mean        std      median        mean   
Segment_Six_Month_CLTV_Pred                                                 
0                             3.291545   2.324635  132.602500  160.041689   
1                             4.373178   2.871065  194.580833  214.859745   
2                             6.237956   4.002962  268.598333  299.377616   
3                            13.249271  17.535714  394.182857  494.602823   

                                        expected_average_profit              \
                                    std                  median        mean   
Segment_Six_Month_CLTV_Pred                                                   
0                            119.869772              147.668700  178.630471   
1                             92.514469              212.553649  234.764206   
2                            231.425721              280.794114  320.913982   
3                            387.263098              414.537786  520.511349   

                                         
                                    std  
Segment_Six_Month_CLTV_Pred              
0                            133.117158  
1                            105.073690  
2                            250.760851  
3                            406.273176